<a href="https://colab.research.google.com/github/ovbystrova/hse_compling/blob/main/hw3/hw3_spellchecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%capture
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt -O data/sents_with_mistakes.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt -O data/correct_sents.txt

In [2]:
%%capture 
!pip install razdel

In [3]:
from collections import Counter, defaultdict
import re
from string import punctuation

from razdel import sentenize
from razdel import tokenize as razdel_tokenize

In [11]:
punctuation += "«»—…“”"
punct = set(punctuation)

bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

corpus = open('data/wiki_data.txt', encoding='utf8').read()
vocab = set(re.findall('\w+', corpus.lower()))

WORDS = Counter(vocab) # Vocab
N = sum(WORDS.values()) # Vocab length

# Задание 1:

Реализуйте алгоритм Symspell - https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция - удаление символа. Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.

In [13]:
bad[0], true[0], corpus[:50]

('Симпатичнейшое шпионское устройство, такой себе гламурный фотоаппарат девушки Бонда - миниатюрная модель камеры Superheadz Clap Camera.',
 'Симпатичнейшее шпионское устройство такой себе гламурный фотоаппарат девушки Бонда миниатюрная модель камеры Superheadz Clap Camera',
 '######Новостройка (Нижегородская область)#########')

In [14]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)] 
    return list(zip(tokens_1, tokens_2))

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

In [17]:
def precompute_dictionary(vocab):
    """
    Generate terms with an edit distance (deletes only) from each dictionary term 
    and add them together with the original term to the dictionary. 
    This has to be done only once during a pre-calculation step.
    """
    dictionary = defaultdict(list)

    for word in vocab:
        deletions = edits1(word)
        for reduced_form in deletions:
            dictionary[reduced_form].append(word)
    return dictionary

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def find_candidates(term, dictionary):
    """
    Generate terms with an edit distance (deletes only) from the input term 
    and search them in the dictionary.
    """
    candidates = [term]
    term_reduced = edits1(term)
    candidates.extend(dictionary[term])
    for reduced in term_reduced:
        candidates.extend(dictionary[reduced])
    return candidates

def find_correction(word): 
    "Находим наиболее вероятное похожее слово"
    candidates = find_candidates(word, dictionary=D)
    return max(candidates, key=P)

In [16]:
D = precompute_dictionary(WORDS)

In [18]:
D['солнц'], D['сонце'], D['лесница'], D['опофеоз']

(['солнце', 'солнцу', 'солнца'], ['солнце'], ['лестница'], [])

In [19]:
find_candidates('сонце', dictionary=D)

['сонце',
 'солнце',
 'донце',
 'конце',
 'монце',
 'соней',
 'сионе',
 'сонче',
 'сонет',
 'слоне',
 'сосне',
 'согне',
 'сконе',
 'сьоне',
 'сотне']

In [20]:
find_correction('сонце'), find_correction('соллнце'), find_correction('опофеоз')

('солнце', 'солнцем', 'апофеоз')

In [21]:
def caltulate_metrics(true, bad):
    """
    Для оценки используем будем использовать три метрики:
    1) процент правильных слов;
    2) процент исправленных ошибок
    3) процент ошибочно исправленных правильных слов
    """
    correct = 0
    total = 0
    total_mistaken = 0
    mistaken_fixed = 0
    total_correct = 0
    correct_broken = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], WORDS):
                predicted = cashed.get(pair[1], find_correction(pair[1]))
            else:
                predicted = pair[1]
            # чтобы два раза не исправлять одно и тоже слово - закешируем его
            # перед тем как считать исправление проверим нет ли его в кеше
            cashed[pair[0]] = predicted       
            if predicted == pair[0]:
                correct += 1
            total += 1
            
            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] !=  predicted:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == predicted:
                    mistaken_fixed += 1

    print('Процент правильных слов: ', correct/total)
    print('Процент исправленных ошибок: ', mistaken_fixed/total_mistaken)
    print('Процент ошибочно исправленных правильных слов: ', correct_broken/total_correct)

In [22]:
caltulate_metrics(true=true, bad=bad)

Процент правильных слов:  0.8652347652347653
Процент исправленных ошибок:  0.30775134305448965
Процент ошибочно исправленных правильных слов:  0.0513380039049041


На данном этапе получилось исправить 30% ошибок, но зато мы меньше делаем исправления там где не надо (чем на семинаре). Что можно доработвать: сделать больше словарь, использовать более сложную функцию при выборе исправления из кандидатов.

# Задание 2.
 Добавьте к полученному алгоритму исправления (Symspell) триграммную модель и проверьте, улучшает ли она качество. Триграммную модель нужно вставить туда, где у вас выбирается один из нескольких кандидатов на исправление.

In [23]:
def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [24]:
corpus_wiki = [['<start>', '<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]

unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus_wiki:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence, 2))
    trigrams.update(ngrammer(sentence, 3))

In [26]:
def calculate_metrics_with_trigramms(true, bad, WORDS=WORDS, D=D, 
                                     bigrams=bigrams, trigrams=trigrams):
    """
    Для оценки используем будем использовать три метрики:
    1) процент правильных слов;
    2) процент исправленных ошибок
    3) процент ошибочно исправленных правильных слов
    """
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0
    total_correct = 0
    correct_broken = 0
    total = 0
    correct = 0

    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])  
        word_pairs = [('<start>', '<start>')] + word_pairs
        pred_sent = []
        for j in range(2, len(word_pairs)):       
            pred = None       
            if not predict_mistaken(word_pairs[j][1], WORDS):
                pred = word_pairs[j][1]    
            else:
                predicted = find_candidates(word_pairs[j][1], dictionary=D)
                prev_word = word_pairs[j-2][1] + ' ' + word_pairs[j-1][1]    
                if prev_word not in bigrams:
                    pred = max(predicted, key=P)
                else:
                    lm_predicted = []
                    for word in predicted:
                        trigram = ' '.join([prev_word, word])
                        lm_predicted.append((word, ((trigrams[trigram]/bigrams[prev_word]))))               
                    if lm_predicted:
                        pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
            if pred is None:
                pred = word_pairs[j][1] 
            if pred == word_pairs[j][0]:
                correct += 1
            else:
                mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
            total += 1        
            if word_pairs[j][0] == word_pairs[j][1]:
                total_correct += 1
                if word_pairs[j][0] !=  pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if word_pairs[j][0] == pred:
                    mistaken_fixed += 1
    print('Процент правильных слов: ', correct/total)
    print('Процент исправленных ошибок: ', mistaken_fixed/total_mistaken)
    print('Процент ошибочно исправленных правильных слов: ', correct_broken/total_correct)

In [27]:
calculate_metrics_with_trigramms(true, bad)

Процент правильных слов:  0.890367275126457
Процент исправленных ошибок:  0.21610738255033557
Процент ошибочно исправленных правильных слов:  0.049467002036172


 Еще немного уменьшился процент ошибочно исправленных правильных слов, но вместе с ним и упал процент исправленных ошибок. В качестве возможных улучшений сюда можно отнести пополнение словаря/корпуса. 